# 3 Classificação e previsão 
## 3.1) Quanto maior os gastos e investimentos com a educação, menor será a taxa de analfabetismo?

In [69]:
import pandas as pd

## 3.1.1) Carga e transformação da massa bruta de todos os dados.

In [334]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

indicadores = ['UIS.LP.AG15T24', 'UIS.ILLPOP.AG25T64', 'UIS.LP.AG65',
               'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 'UIS.XSPENDP.FDPUB.FNNONS', 
               'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GB.ZS', 'SE.XPD.TOTL.GD.ZS']

anos = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

colunas_classificado = ['Pais', 'Indicador', '1998-1999', '1999-2000', '2000-2001', '2001-2002', 
                        '2002-2003', '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008', 
                        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013']

df_classificado_todos = pd.DataFrame(columns=colunas_classificado)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)
        
    for index, value in indicador.items():            
        valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[0] - df_1998_a_2013['1998'].iloc[0] > 0 else 0
        valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[0] - df_1998_a_2013['1999'].iloc[0] > 0 else 0
        valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[0] - df_1998_a_2013['2000'].iloc[0] > 0 else 0
        valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[0] - df_1998_a_2013['2001'].iloc[0] > 0 else 0
        valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[0] - df_1998_a_2013['2002'].iloc[0] > 0 else 0
        valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[0] - df_1998_a_2013['2003'].iloc[0] > 0 else 0
        valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[0] - df_1998_a_2013['2004'].iloc[0] > 0 else 0
        valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[0] - df_1998_a_2013['2005'].iloc[0] > 0 else 0
        valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[0] - df_1998_a_2013['2006'].iloc[0] > 0 else 0
        valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[0] - df_1998_a_2013['2007'].iloc[0] > 0 else 0
        valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[0] - df_1998_a_2013['2008'].iloc[0] > 0 else 0
        valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[0] - df_1998_a_2013['2009'].iloc[0] > 0 else 0
        valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[0] - df_1998_a_2013['2010'].iloc[0] > 0 else 0
        valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[0] - df_1998_a_2013['2011'].iloc[0] > 0 else 0
        valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[0] - df_1998_a_2013['2012'].iloc[0] > 0 else 0

        valores = [pais, value, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                   valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                   valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012, 
                   valor_2012_2013]

        df_auxiliar = pd.DataFrame([valores], columns=colunas_classificado)
        df_auxiliar.index = range(df_auxiliar.shape[0])
        df_classificado_todos = pd.concat([df_auxiliar, df_classificado_todos])
        
    
df_classificado_todos = df_classificado_todos.sort_values(['Pais'], ascending=[True])
df_classificado_todos.index = range(df_classificado_todos.shape[0])

## 3.1.1.1) Validação da eficiência da previsão

In [349]:
#6) Aplicar algoritmo de classificação e previsão

#df_classificado_todos.describe().round(2)
df_periodo_datas = df_classificado_todos.iloc[:, 2:18]
#df_classificado.to_csv('resultado_teste.csv', index = False)



AttributeError: 'stdout' is not a valid function for 'Series' object

## 3.1.2) Carga e transformação da massa bruta dos dados com pelo menos 50% dos valores preenchidos.

Isso reduzirá a quantidade de elementos para 351. Na solução 3.1.1, sem a restrição de 50% do preenchimento, foram utilizados 1953 elementos.

In [339]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

indicadores = ['UIS.LP.AG15T24', 'UIS.ILLPOP.AG25T64', 'UIS.LP.AG65',
               'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 'UIS.XSPENDP.FDPUB.FNNONS', 
               'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GB.ZS', 'SE.XPD.TOTL.GD.ZS']

anos = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

colunas_classificado = ['Pais', 'Indicador', '1998-1999', '1999-2000', '2000-2001', '2001-2002', 
                        '2002-2003', '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008', 
                        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013']

df_classificado = pd.DataFrame(columns=colunas_classificado)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    media_valores_invalidos = df_1998_a_2013.isna().mean(axis=1).mean()
    
    if media_valores_invalidos < 0.5:
        
        df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)
        
        for index, value in indicador.items():            
            valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[0] - df_1998_a_2013['1998'].iloc[0] > 0 else 0
            valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[0] - df_1998_a_2013['1999'].iloc[0] > 0 else 0
            valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[0] - df_1998_a_2013['2000'].iloc[0] > 0 else 0
            valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[0] - df_1998_a_2013['2001'].iloc[0] > 0 else 0
            valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[0] - df_1998_a_2013['2002'].iloc[0] > 0 else 0
            valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[0] - df_1998_a_2013['2003'].iloc[0] > 0 else 0
            valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[0] - df_1998_a_2013['2004'].iloc[0] > 0 else 0
            valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[0] - df_1998_a_2013['2005'].iloc[0] > 0 else 0
            valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[0] - df_1998_a_2013['2006'].iloc[0] > 0 else 0
            valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[0] - df_1998_a_2013['2007'].iloc[0] > 0 else 0
            valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[0] - df_1998_a_2013['2008'].iloc[0] > 0 else 0
            valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[0] - df_1998_a_2013['2009'].iloc[0] > 0 else 0
            valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[0] - df_1998_a_2013['2010'].iloc[0] > 0 else 0
            valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[0] - df_1998_a_2013['2011'].iloc[0] > 0 else 0
            valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[0] - df_1998_a_2013['2012'].iloc[0] > 0 else 0
            
            valores = [pais, value, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                       valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                       valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012, 
                       valor_2012_2013]

            df_auxiliar = pd.DataFrame([valores], columns=colunas_classificado)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_classificado = pd.concat([df_auxiliar, df_classificado])
        
df_classificado = df_classificado.sort_values(['Pais'], ascending=[True])
df_classificado.index = range(df_classificado.shape[0])

## 3.1.2.1) Validação da eficiência da previsão

In [340]:
df_classificado.describe().round(2)

,Pais,Indicador,1998-1999,1999-2000,2000-2001,2001-2002,2002-2003,2003-2004,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013
count,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351,351
unique,39,9,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,Switzerland,UIS.LP.AG15T24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
freq,9,39,261,216,252,216,198,207,189,252,207,252,225,216,180,198,243
